In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'Pernambuco - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [8]:
data = pd.read_csv('2003_mo_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego,Pernambuco - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.297418,34.266080,8.237381e+07,5.574893e+06,8.646979,7.571061e+07,0.688685,8.294029,99.743
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.298912,34.126272,8.248102e+07,5.580364e+06,8.650960,7.575437e+07,0.689009,8.288006,86.997
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.300346,34.414272,8.258823e+07,5.585835e+06,8.654941,7.579812e+07,0.689334,8.281983,83.033
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.301729,34.576094,8.269544e+07,5.591305e+06,8.658922,7.584188e+07,0.689658,8.275960,82.302
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.302368,34.890197,8.280265e+07,5.596776e+06,8.662903,7.588563e+07,0.689983,8.269937,80.217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.478890,26.779678,NaN,NaN,NaN,NaN,NaN,NaN,164.137
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.478683,26.996501,NaN,NaN,NaN,NaN,NaN,NaN,168.356
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.476012,26.919241,NaN,NaN,NaN,NaN,NaN,NaN,172.421
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.473486,27.026250,NaN,NaN,NaN,NaN,NaN,NaN,165.411


In [9]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.011454,-1.188910,-1.754154,-0.181030,-2.511326,-2.321118,-2.100972,-0.864836
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.964614,-1.195377,-1.734705,-0.148206,-2.462275,-2.280399,-2.067858,-0.866889
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.919667,-1.182056,-1.715257,-0.115382,-2.413223,-2.239680,-2.034743,-0.868942
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.876288,-1.174571,-1.695809,-0.082559,-2.364172,-2.198961,-2.001628,-0.870995
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.856283,-1.160042,-1.676361,-0.049735,-2.315120,-2.158242,-1.968514,-0.873048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.720800,-1.281757,1.016740,-1.416488,-0.070259,0.355961,1.324101,1.183158
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.802101,-1.271596,1.004554,-1.402870,-0.093306,0.336941,1.314733,1.181904
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.813469,-1.268136,0.992368,-1.389252,-0.116354,0.317922,1.305366,1.180650
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.843043,-1.270527,0.980182,-1.375634,-0.139402,0.298902,1.295999,1.179397


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      76.341
1      65.040
2      88.306
3      74.323
4      78.114
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Pernambuco - Consumo de Cimento (t), Length: 240, dtype: float64

In [11]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.011454,-1.188910,-1.754154,-0.181030,-2.511326,-2.321118,-2.100972,-0.864836
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.964614,-1.195377,-1.734705,-0.148206,-2.462275,-2.280399,-2.067858,-0.866889
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.919667,-1.182056,-1.715257,-0.115382,-2.413223,-2.239680,-2.034743,-0.868942
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.876288,-1.174571,-1.695809,-0.082559,-2.364172,-2.198961,-2.001628,-0.870995
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.856283,-1.160042,-1.676361,-0.049735,-2.315120,-2.158242,-1.968514,-0.873048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.354413,-0.165538,1.170965,-1.488331,0.445555,0.786479,1.391411,1.357017
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.374652,-0.236996,1.172520,-1.494202,0.438851,0.781348,1.390568,1.349817
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.400582,-0.309267,1.174075,-1.500073,0.432147,0.776217,1.389726,1.342617
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.422300,-0.382346,1.175630,-1.505943,0.425442,0.771086,1.388883,1.335417


In [12]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0       76.341
1       65.040
2       88.306
3       74.323
4       78.114
        ...   
157    143.388
158    188.227
159    143.049
160    148.068
161    133.872
Name: Pernambuco - Consumo de Cimento (t), Length: 162, dtype: float64

In [13]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego
162,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,0.502302,-0.514627,1.178739,-1.517685,0.412033,0.760824,1.387198,1.321016
163,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,0.558872,-0.579807,1.180294,-1.523555,0.405329,0.755693,1.386356,1.313816
164,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,0.618009,-0.637869,1.181849,-1.529426,0.398625,0.750563,1.385514,1.306616
165,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,0.677851,-0.700816,1.183404,-1.535296,0.391920,0.745432,1.384671,1.299416
166,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,0.735370,-0.762356,1.184958,-1.541167,0.385216,0.740301,1.383829,1.292215
167,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,0.805344,-0.819696,1.186513,-1.547038,0.378512,0.735170,1.382986,1.285015
168,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,0.862523,-0.868227,1.188068,-1.552908,0.371807,0.730039,1.382144,1.277815
169,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,0.919373,-0.920978,1.180899,-1.549484,0.348413,0.709961,1.382771,1.270658
170,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,0.971509,-0.964738,1.173731,-1.546060,0.325019,0.689882,1.383399,1.263502
171,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.026374,-1.008836,1.166562,-1.542635,0.301625,0.669804,1.384026,1.256345


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    131.464
163    151.370
164    154.276
165    153.331
166    152.600
167    136.567
168    144.855
169    126.352
170    138.148
171    129.978
172    109.083
173    147.440
174    132.500
175    158.533
176    133.813
177    155.827
178    146.197
179    122.278
180    149.983
181    123.483
182    125.104
183    129.633
184    138.233
185    109.657
186    128.809
187    140.408
188    134.634
189    189.040
190    166.406
191    155.456
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [15]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [27]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
#                         validation_split=0.07,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [28]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [29]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[531077790, 1920377228, 1623047814, 2125771211, 3115464542, 3429880134, 4225001622, 3132872968, 2942341223, 2078227689, 3618974338, 2030182595, 2493022402, 1800086960, 3827629207, 2545598500, 1643889302, 1486655283, 3974757553, 2899745877, 3305859953, 398547712, 1158171961, 95807413, 3395698231]


Step: 0 ___________________________________________
val_loss: 278.6745910644531
winner_seed: 531077790


Step: 1 ___________________________________________
val_loss: 242.3434600830078
winner_seed: 1920377228


Step: 2 ___________________________________________
val_loss: 239.02346801757812
winner_seed: 1623047814


Step: 3 ___________________________________________
val_loss: 304.3401184082031


Step: 4 ___________________________________________
val_loss: 249.42066955566406


Step: 5 ___________________________________________
val_loss: 247.76885986328125


Step: 6 ___________________________________________
val_loss: 265.76995849609375


Step: 7 ___________________________________________


2023-10-14 17:59:51.961607: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 296.44647216796875


Step: 22 ___________________________________________
val_loss: 1125.219482421875


Step: 23 ___________________________________________
val_loss: 223.82411193847656


Step: 24 ___________________________________________
val_loss: 222.43142700195312


final_seed: 3618974338


In [30]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
6/6 [==============================] - 1s 23ms/step - loss: 27346.1074 - val_loss: 42323.7578
Epoch 2/10000
6/6 [==============================] - 0s 6ms/step - loss: 33149.3945 - val_loss: 57040.7695
Epoch 3/10000
6/6 [==============================] - 0s 5ms/step - loss: 24325.5469 - val_loss: 28751.0020
Epoch 4/10000
6/6 [==============================] - 0s 5ms/step - loss: 17796.5176 - val_loss: 15321.7305
Epoch 5/10000
6/6 [==============================] - 0s 5ms/step - loss: 6841.9268 - val_loss: 23482.0234
Epoch 6/10000
6/6 [==============================] - 0s 5ms/step - loss: 16825.3496 - val_loss: 16768.5137
Epoch 7/10000
6/6 [==============================] - 0s 5ms/step - loss: 8435.0449 - val_loss: 1134.7185
Epoch 8/10000
6/6 [==============================] - 0s 5ms/step - loss: 637.0102 - val_loss: 438.0541
Epoch 9/10000
6/6 [==============================] - 0s 5ms/step - loss: 610.8011 - val_loss: 957.7118
Epoch 10/10000
6/6 [===========================

6/6 [==============================] - 0s 5ms/step - loss: 370.4028 - val_loss: 356.5424
Epoch 80/10000
6/6 [==============================] - 0s 5ms/step - loss: 330.6968 - val_loss: 425.8486
Epoch 81/10000
6/6 [==============================] - 0s 6ms/step - loss: 383.5642 - val_loss: 756.8238
Epoch 82/10000
6/6 [==============================] - 0s 5ms/step - loss: 530.5408 - val_loss: 594.7144
Epoch 83/10000
6/6 [==============================] - 0s 5ms/step - loss: 395.0948 - val_loss: 545.9412
Epoch 84/10000
6/6 [==============================] - 0s 6ms/step - loss: 387.4770 - val_loss: 436.2207
Epoch 85/10000
6/6 [==============================] - 0s 5ms/step - loss: 306.2565 - val_loss: 558.6293
Epoch 86/10000
6/6 [==============================] - 0s 6ms/step - loss: 379.1378 - val_loss: 415.5315
Epoch 87/10000
6/6 [==============================] - 0s 6ms/step - loss: 328.7350 - val_loss: 485.4379
Epoch 88/10000
6/6 [==============================] - 0s 6ms/step - loss: 377.1

6/6 [==============================] - 0s 6ms/step - loss: 392.4398 - val_loss: 422.5421
Epoch 158/10000
6/6 [==============================] - 0s 6ms/step - loss: 284.3801 - val_loss: 393.7601
Epoch 159/10000
6/6 [==============================] - 0s 6ms/step - loss: 302.7841 - val_loss: 568.5237
Epoch 160/10000
6/6 [==============================] - 0s 5ms/step - loss: 392.3465 - val_loss: 895.7908
Epoch 161/10000
6/6 [==============================] - 0s 6ms/step - loss: 411.9248 - val_loss: 737.8949
Epoch 162/10000
6/6 [==============================] - 0s 7ms/step - loss: 323.2833 - val_loss: 737.8433
Epoch 163/10000
6/6 [==============================] - 0s 6ms/step - loss: 381.4066 - val_loss: 635.9763
Epoch 164/10000
6/6 [==============================] - 0s 6ms/step - loss: 358.3741 - val_loss: 461.9025
Epoch 165/10000
6/6 [==============================] - 0s 5ms/step - loss: 338.2469 - val_loss: 449.9329
Epoch 166/10000
6/6 [==============================] - 0s 6ms/step - lo

6/6 [==============================] - 0s 6ms/step - loss: 273.0449 - val_loss: 457.2561
Epoch 236/10000
6/6 [==============================] - 0s 6ms/step - loss: 255.6572 - val_loss: 362.4608
Epoch 237/10000
6/6 [==============================] - 0s 7ms/step - loss: 278.0210 - val_loss: 435.6871
Epoch 238/10000
6/6 [==============================] - 0s 7ms/step - loss: 325.3055 - val_loss: 469.0598
Epoch 239/10000
6/6 [==============================] - 0s 6ms/step - loss: 252.3091 - val_loss: 544.9775
Epoch 240/10000
6/6 [==============================] - 0s 6ms/step - loss: 314.3940 - val_loss: 428.1109
Epoch 241/10000
6/6 [==============================] - 0s 6ms/step - loss: 274.4034 - val_loss: 459.5818
Epoch 242/10000
6/6 [==============================] - 0s 7ms/step - loss: 347.8919 - val_loss: 694.9657
Epoch 243/10000
6/6 [==============================] - 0s 6ms/step - loss: 382.0259 - val_loss: 504.8380
Epoch 244/10000
6/6 [==============================] - 0s 6ms/step - lo

6/6 [==============================] - 0s 5ms/step - loss: 405.8154 - val_loss: 444.1183
Epoch 314/10000
6/6 [==============================] - 0s 6ms/step - loss: 286.8599 - val_loss: 302.4537
Epoch 315/10000
6/6 [==============================] - 0s 6ms/step - loss: 271.8092 - val_loss: 360.5768
Epoch 316/10000
6/6 [==============================] - 0s 5ms/step - loss: 275.7972 - val_loss: 316.4964
Epoch 317/10000
6/6 [==============================] - 0s 6ms/step - loss: 309.9977 - val_loss: 760.9093
Epoch 318/10000
6/6 [==============================] - 0s 6ms/step - loss: 351.3087 - val_loss: 399.2849
Epoch 319/10000
6/6 [==============================] - 0s 6ms/step - loss: 289.0125 - val_loss: 656.7007
Epoch 320/10000
6/6 [==============================] - 0s 6ms/step - loss: 391.3496 - val_loss: 375.8720
Epoch 321/10000
6/6 [==============================] - 0s 6ms/step - loss: 300.3432 - val_loss: 364.9591
Epoch 322/10000
6/6 [==============================] - 0s 5ms/step - lo

...

6/6 [==============================] - 0s 5ms/step - loss: 232.4655 - val_loss: 410.0109
Epoch 1398/10000
6/6 [==============================] - 0s 5ms/step - loss: 212.8802 - val_loss: 422.3011
Epoch 1399/10000
6/6 [==============================] - 0s 5ms/step - loss: 247.6990 - val_loss: 460.1610
Epoch 1400/10000
6/6 [==============================] - 0s 5ms/step - loss: 219.7498 - val_loss: 565.0684
Epoch 1401/10000
6/6 [==============================] - 0s 6ms/step - loss: 309.5847 - val_loss: 842.2578
Epoch 1402/10000
6/6 [==============================] - 0s 7ms/step - loss: 290.4047 - val_loss: 480.1142
Epoch 1403/10000
6/6 [==============================] - 0s 5ms/step - loss: 211.6754 - val_loss: 489.6463
Epoch 1404/10000
6/6 [==============================] - 0s 6ms/step - loss: 254.0742 - val_loss: 495.4270
Epoch 1405/10000
6/6 [==============================] - 0s 6ms/step - loss: 223.0714 - val_loss: 401.0359
Epoch 1406/10000
6/6 [==============================] - 0s 5ms/

6/6 [==============================] - 0s 5ms/step - loss: 347.5450 - val_loss: 276.7278
Epoch 1475/10000
6/6 [==============================] - 0s 5ms/step - loss: 228.0770 - val_loss: 673.8903
Epoch 1476/10000
6/6 [==============================] - 0s 6ms/step - loss: 300.5188 - val_loss: 339.3644
Epoch 1477/10000
6/6 [==============================] - 0s 5ms/step - loss: 262.8594 - val_loss: 333.6434
Epoch 1478/10000
6/6 [==============================] - 0s 5ms/step - loss: 288.3243 - val_loss: 391.5688
Epoch 1479/10000
6/6 [==============================] - 0s 5ms/step - loss: 287.4507 - val_loss: 477.9944
Epoch 1480/10000
6/6 [==============================] - 0s 5ms/step - loss: 303.9768 - val_loss: 321.2682
Epoch 1481/10000
6/6 [==============================] - 0s 6ms/step - loss: 231.7488 - val_loss: 386.8142
Epoch 1482/10000
6/6 [==============================] - 0s 6ms/step - loss: 349.6195 - val_loss: 449.2727
Epoch 1483/10000
6/6 [==============================] - 0s 5ms/

6/6 [==============================] - 0s 5ms/step - loss: 259.9221 - val_loss: 555.5159
Epoch 1552/10000
6/6 [==============================] - 0s 5ms/step - loss: 306.5136 - val_loss: 339.7154
Epoch 1553/10000
6/6 [==============================] - 0s 5ms/step - loss: 263.2043 - val_loss: 715.9855
Epoch 1554/10000
6/6 [==============================] - 0s 7ms/step - loss: 326.1284 - val_loss: 412.2155
Epoch 1555/10000
6/6 [==============================] - 0s 6ms/step - loss: 281.7176 - val_loss: 516.9923
Epoch 1556/10000
6/6 [==============================] - 0s 5ms/step - loss: 251.9067 - val_loss: 412.4727
Epoch 1557/10000
6/6 [==============================] - 0s 5ms/step - loss: 264.5924 - val_loss: 367.0853
Epoch 1558/10000
6/6 [==============================] - 0s 5ms/step - loss: 245.7843 - val_loss: 474.5104
Epoch 1559/10000
6/6 [==============================] - 0s 5ms/step - loss: 263.0805 - val_loss: 439.6995
Epoch 1560/10000
6/6 [==============================] - 0s 5ms/

6/6 [==============================] - 0s 5ms/step - loss: 246.1198 - val_loss: 363.5664
Epoch 1629/10000
6/6 [==============================] - 0s 5ms/step - loss: 239.2792 - val_loss: 418.6150
Epoch 1630/10000
6/6 [==============================] - 0s 5ms/step - loss: 319.7807 - val_loss: 371.5014
Epoch 1631/10000
6/6 [==============================] - 0s 5ms/step - loss: 237.5169 - val_loss: 393.3523
Epoch 1632/10000
6/6 [==============================] - 0s 5ms/step - loss: 232.5301 - val_loss: 376.5294
Epoch 1633/10000
6/6 [==============================] - 0s 5ms/step - loss: 251.5998 - val_loss: 367.4741
Epoch 1634/10000
6/6 [==============================] - 0s 5ms/step - loss: 239.2186 - val_loss: 551.0356
Epoch 1635/10000
6/6 [==============================] - 0s 6ms/step - loss: 313.4775 - val_loss: 326.2105
Epoch 1636/10000
6/6 [==============================] - 0s 5ms/step - loss: 225.1655 - val_loss: 441.5261
Epoch 1637/10000
6/6 [==============================] - 0s 5ms/

6/6 [==============================] - 0s 5ms/step - loss: 238.4120 - val_loss: 351.7244
Epoch 1706/10000
6/6 [==============================] - 0s 5ms/step - loss: 219.0640 - val_loss: 809.2628
Epoch 1707/10000
6/6 [==============================] - 0s 5ms/step - loss: 275.1613 - val_loss: 356.7110
Epoch 1708/10000
6/6 [==============================] - 0s 5ms/step - loss: 211.4200 - val_loss: 327.5567
Epoch 1709/10000
6/6 [==============================] - 0s 6ms/step - loss: 237.0913 - val_loss: 665.6968
Epoch 1710/10000
6/6 [==============================] - 0s 6ms/step - loss: 266.5270 - val_loss: 486.3304
Epoch 1711/10000
6/6 [==============================] - 0s 5ms/step - loss: 243.5673 - val_loss: 372.8746
Epoch 1712/10000
6/6 [==============================] - 0s 6ms/step - loss: 225.9658 - val_loss: 355.7498
Epoch 1713/10000
6/6 [==============================] - 0s 5ms/step - loss: 217.4989 - val_loss: 874.7770
Epoch 1714/10000
6/6 [==============================] - 0s 5ms/

In [31]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [32]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,162.714401,162.714386,162.714386,162.714355,162.714355,162.714294,162.279633,158.672943,158.628662,147.997742,31.130033,132.934448,12.686081,5.928772,12.686081,12.61084,12.427887,4.457733,1.9319,2.007286,1.924217,1.923828,1.923439,1.923431,1.923431,-12.451607,-12.45163,-24.349281,-31.062111,-31.065659
Target,131.464,151.37,154.276,153.331,152.6,136.567,144.855,126.352,138.148,129.978,109.083,147.44,132.5,158.533,133.813,155.827,146.197,122.278,149.983,123.483,125.104,129.633,138.233,109.657,128.809,140.408,134.634,189.04,166.406,155.456
Error,31.250397,11.344391,8.438385,9.383362,10.114349,26.147293,17.424637,32.320946,20.480667,18.019745,77.952965,14.505554,119.813919,152.604233,121.126923,143.216156,133.769119,117.820267,148.051102,121.475716,123.179779,127.709167,136.30957,107.733566,126.885574,152.859619,147.085632,213.389282,197.468109,186.521652


In [33]:
display(mae)
display(mape)

95.14674

0.6774221

In [34]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [35]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1767.9297]] - Target[1675.4640000000002]| =  Error: [[92.4657]]; MAPE:[[0.05518812]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[72.431496]] - Target[1625.241]| =  Error: [[1552.8094]]; MAPE:[[0.95543337]]
1/1 [==============================] - 0s 17ms/step
Ano-5: |Prediction[[-109.456856]] - Target[914.753]| =  Error: [[1024.2098]]; MAPE:[[1.1196573]]


[array([[92.4657]], dtype=float32),
 array([[1552.8094]], dtype=float32),
 array([[1024.2098]], dtype=float32)]

889.8283

0.71009284